# Segmenting and Clustering Neighborhoods in Toronto
## Peer-graded Assignment

We start by scraping the following Wikipedia page: 
<a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M">Canadian Postal Codes with M</a>. 

In order to transform the above table in a pandas dataframe, we will use <b>BeautifulSoup</b>

## Scraping the web page (Part 1)

### Getting the HTML page 
#### Getting the required libraries 


In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests

In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
!pip install folium
import folium

     |████████████████████████████████| 92kB 4.4MB/s eta 0:00:01


In [3]:
from bs4 import BeautifulSoup as soup
import requests 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url).text

In [5]:
html = soup(page, 'lxml')

In [6]:
#print (html.prettify()) #uncomment if you want to print out the html page

In [7]:
table = html.find('table', class_ = 'wikitable sortable') #store the table
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [8]:
headers = table.tbody.tr.find_all('th') #store the headers
headers

[<th>Postcode</th>, <th>Borough</th>, <th>Neighbourhood
 </th>]

In [9]:
column_name  = []
for n in range(len(headers)):
    header = headers[n].text.strip()
    column_name.append(header)           

In [10]:
column_name

['Postcode', 'Borough', 'Neighbourhood']

In [11]:
rows = table.tbody.find_all('tr') #find the rows

In [13]:
# create an empty dataframe with the header just found 
df =  pd.DataFrame(columns = column_name)
df

,Postcode,Borough,Neighbourhood


In [14]:
for n in range(1, len(rows)):
    row = rows[n].find_all('td')
    postcode = row[0].text.strip() #we strip to avoid new pages signs
    borough = row[1].text.strip()
    if borough == 'Not assigned':
        continue 
    neighbourhood = row[2].text.strip()
    if neighbourhood == 'Not assigned':
        neighbourhood = borough
      
    df = df.append({'Postcode': postcode,
                    'Borough': borough,
                    'Neighbourhood': neighbourhood}, 
                    ignore_index = True)

In [15]:
df.rename(columns = {'Neighbourhood': 'Neighborhood'}, inplace = True)
df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [16]:
# rename the columns for better suitability with the code to follow
codes = df.groupby(['Postcode', 'Borough'],as_index=False).agg(', '.join)
codes.rename(columns = {'Postcode':'PostalCode'}, inplace = True)

In [17]:
codes.shape

(103, 3)

## Download the coordinates
We take the coordinates from the csv file provided in the assignment. 

In [18]:
!wget -q -O 'geospatial_data.csv' https://cocl.us/Geospatial_data

print('Data downloaded!')

Data downloaded!


In [19]:
geo_data = pd.read_csv('geospatial_data.csv')
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
#we rename
geo_data.rename(columns = {'Postal Code': 'PostalCode'}, inplace = True)

In [22]:
# and we merge
df_geo = pd.merge(codes, geo_data, on="PostalCode")

In [23]:
df_geo.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [24]:
#check the shape
df_geo.shape

(103, 5)

In the following we will only consider neighborhoods which have **York** in their name

In [25]:
york = df_geo[df_geo['Borough'].str.contains('York')]
latitude = york['Latitude'].mean()
longitude = york['Longitude'].mean()

## Getting the data from FourSquare (Part 2)

In [ ]:
CLIENT_ID = '' #removed for privacy # your Foursquare ID
CLIENT_SECRET = '' #  #removed for privacy your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [27]:
LIMIT = 90
radius = 400

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
#store the venues in a dataframe
york_venues = getNearbyVenues(names=york['Neighborhood'],
                                   latitudes=york['Latitude'],
                                   longitudes=york['Longitude']
                                  )

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
Leaside
Thorncliffe Park
East Toronto
Bedford Park, Lawrence Manor East
Lawrence Heights, Lawrence Manor
Glencairn
Humewood-Cedarvale
Caledonia-Fairbanks
Downsview, North Park, Upwood Park
Del Ray, Keelesdale, Mount Dennis, Silverthorn
The Junction North, Runnymede
Humber Summit
Emery, Humberlea
Weston


In [31]:
york_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,New York Fries,43.803664,-79.363905,Fast Food Restaurant
2,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
3,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
4,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run


In [33]:
york_venues.groupby('Neighborhood').count() #how many venues did we find?

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
"CFB Toronto, Downsview East",2,2,2,2,2,2
Caledonia-Fairbanks,5,5,5,5,5,5
"Del Ray, Keelesdale, Mount Dennis, Silverthorn",4,4,4,4,4,4
Don Mills North,5,5,5,5,5,5
Downsview Central,4,4,4,4,4,4
Downsview Northwest,4,4,4,4,4,4


## Clustering (Part 3)
We start the clustering part of our work. We begin with one hot encoding.

In [47]:
# one hot encoding
york_onehot = pd.get_dummies(york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
york_onehot['Neighborhood'] = york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [york_onehot.columns[-1]] + list(york_onehot.columns[:-1])
york_onehot = york_onehot[fixed_columns]

york_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,...,Theater,Toy / Game Store,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
york_grouped = york_onehot.groupby('Neighborhood').mean().reset_index()
york_grouped.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,Bank,...,Theater,Toy / Game Store,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,"Bathurst Manor, Downsview North, Wilson Heights",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.055556,...,0.0,0.0,0.0,0.0,0.055556,0.0,0.0,0.0,0.0,0.0
1,Bayview Village,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.250000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Bedford Park, Lawrence Manor East",0.0,0.0,0.043478,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,"CFB Toronto, Downsview East",0.0,0.5,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,Caledonia-Fairbanks,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.2,0.0


In [36]:
num_top_venues = 10

for hood in york_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = york_grouped[york_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Downsview North, Wilson Heights----
                 venue  freq
0          Coffee Shop  0.11
1   Frozen Yogurt Shop  0.06
2        Shopping Mall  0.06
3       Sandwich Place  0.06
4           Restaurant  0.06
5        Deli / Bodega  0.06
6          Bridal Shop  0.06
7          Supermarket  0.06
8     Sushi Restaurant  0.06
9  Fried Chicken Joint  0.06


----Bayview Village----
                       venue  freq
0                       Bank  0.25
1         Chinese Restaurant  0.25
2                       Café  0.25
3        Japanese Restaurant  0.25
4         Miscellaneous Shop  0.00
5                Men's Store  0.00
6              Metro Station  0.00
7         Mexican Restaurant  0.00
8  Middle Eastern Restaurant  0.00
9          Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                  venue  freq
0    Italian Restaurant  0.09
1           Coffee Shop  0.09
2         Grocery Store  0.04
3        Sandwich Place  0.04
4     Indian Restaur

                  venue  freq
0           Coffee Shop  0.14
1  Caribbean Restaurant  0.14
2        Massage Studio  0.14
3         Metro Station  0.14
4                   Bar  0.14
5    Miscellaneous Shop  0.14
6    Falafel Restaurant  0.14
7                  Park  0.00
8         Movie Theater  0.00
9     Martial Arts Dojo  0.00


----Parkwoods----
                      venue  freq
0      Fast Food Restaurant  0.33
1                      Park  0.33
2         Food & Drink Shop  0.33
3         Accessories Store  0.00
4             Metro Station  0.00
5         Martial Arts Dojo  0.00
6            Massage Studio  0.00
7  Mediterranean Restaurant  0.00
8               Men's Store  0.00
9        Mexican Restaurant  0.00


----Silver Hills, York Mills----
                       venue  freq
0          Martial Arts Dojo   1.0
1          Accessories Store   0.0
2              Luggage Store   0.0
3                   Pharmacy   0.0
4                  Pet Store   0.0
5                       Park   

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = york_grouped['Neighborhood']

for ind in np.arange(york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Pharmacy,Supermarket,Fast Food Restaurant,Diner,Deli / Bodega,Middle Eastern Restaurant,Frozen Yogurt Shop,Pizza Place,Restaurant
1,Bayview Village,Chinese Restaurant,Bank,Japanese Restaurant,Café,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
2,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Greek Restaurant,Cupcake Shop,Pub,Pizza Place,Butcher,Pharmacy,Comfort Food Restaurant,Liquor Store
3,"CFB Toronto, Downsview East",Airport,Park,Yoga Studio,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Curling Ice
4,Caledonia-Fairbanks,Park,Women's Store,Fast Food Restaurant,Market,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop


In [41]:
# set number of clusters
kclusters = 6

york_grouped_clustering = york_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:4] 

array([0, 0, 0, 2], dtype=int32)

In [42]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

york_merged = york

# merge data to add latitude/longitude for each neighborhood
york_merged =york_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


york_merged.dropna(inplace = True)
york_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,M2H,North York,Hillcrest Village,43.803762,-79.363452,0,Golf Course,Pool,Fast Food Restaurant,Mediterranean Restaurant,Athletics & Sports,Dog Run,Convenience Store,Cosmetics Shop,Cupcake Shop,Discount Store
18,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Bakery,Food Court,Japanese Restaurant,Asian Restaurant,Women's Store,Tea Room,Sporting Goods Shop
19,M2K,North York,Bayview Village,43.786947,-79.385975,0,Chinese Restaurant,Bank,Japanese Restaurant,Café,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
20,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,5,Martial Arts Dojo,Yoga Studio,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Curling Ice,Deli / Bodega
21,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,3,Piano Bar,Gym,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Curling Ice


In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(york_merged['Latitude'], york_merged['Longitude'], york_merged['Neighborhood'], york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining the clusters

### First Cluster
The cluster contains a variety of venues of different categories

In [45]:
york_merged.loc[york_merged['Cluster Labels'] == 0, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,North York,0,Golf Course,Pool,Fast Food Restaurant,Mediterranean Restaurant,Athletics & Sports,Dog Run,Convenience Store,Cosmetics Shop,Cupcake Shop,Discount Store
18,North York,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Bakery,Food Court,Japanese Restaurant,Asian Restaurant,Women's Store,Tea Room,Sporting Goods Shop
19,North York,0,Chinese Restaurant,Bank,Japanese Restaurant,Café,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
22,North York,0,Coffee Shop,Ramen Restaurant,Japanese Restaurant,Pizza Place,Restaurant,Sandwich Place,Café,Juice Bar,Pet Store,Hotel
24,North York,0,Discount Store,Grocery Store,Pizza Place,Coffee Shop,Pharmacy,Butcher,Curling Ice,Dim Sum Restaurant,Dessert Shop,Department Store
26,North York,0,Baseball Field,Japanese Restaurant,Gym / Fitness Center,Caribbean Restaurant,Café,Yoga Studio,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
27,North York,0,Coffee Shop,Asian Restaurant,Gym,Beer Store,Restaurant,Japanese Restaurant,Italian Restaurant,Fast Food Restaurant,Bike Shop,Discount Store
28,North York,0,Coffee Shop,Pharmacy,Supermarket,Fast Food Restaurant,Diner,Deli / Bodega,Middle Eastern Restaurant,Frozen Yogurt Shop,Pizza Place,Restaurant
29,North York,0,Coffee Shop,Caribbean Restaurant,Miscellaneous Shop,Falafel Restaurant,Metro Station,Massage Studio,Bar,Yoga Studio,Deli / Bodega,Dessert Shop
31,North York,0,Park,Hotel,Shopping Mall,Bank,Grocery Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Curling Ice


### Second Cluster
This cluster contains only one borough. A crucial difference from the other boroghs is the absence of a café or a bar or a coffee shop

In [93]:
york_merged.loc[york_merged['Cluster Labels'] == 1, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
97,North York,1.0,Baseball Field,Yoga Studio,Falafel Restaurant,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant


### Third cluster
This cluster contains only 4 boroughs, which have parks and general stores, but they seem to miss restaurants or alike.

In [94]:
york_merged.loc[york_merged['Cluster Labels'] == 2, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,North York,2.0,Convenience Store,Park,Bank,Yoga Studio,Event Space,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store,Dessert Shop
40,East York,2.0,Coffee Shop,Park,Convenience Store,Pizza Place,Fried Chicken Joint,Dog Run,Construction & Landscaping,Furniture / Home Store,Frozen Yogurt Shop,Cosmetics Shop
72,North York,2.0,Japanese Restaurant,Pizza Place,Pub,Park,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store
96,North York,2.0,Empanada Restaurant,Pizza Place,Yoga Studio,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Deli / Bodega,Department Store,Dessert Shop


### Fourth cluster
This cluster is similar to the third one, but it also have some food-related venues. 

In [46]:
york_merged.loc[york_merged['Cluster Labels'] == 3, york_merged.columns[[1] + list(range(5, york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,North York,3,Piano Bar,Gym,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Curling Ice
